In [ ]:
import pandas as pd
import numpy as np

from mlpckg.util_method import *
from mlpckg.AOO import *
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# select_key() argument ("1.novem/ 2.trans/ 3.bipun/ 4.paren ")
# select_key() return (target_species,key)

num = 1
target_species,key= select_key(num)

In [ ]:
shap_var = pd.read_csv("./Data/3rd_manipulated_data/shap_values/"+target_species+"shap_values_top15.csv")
shap_var = list(shap_var.name.values)

In [ ]:
df_train = pd.read_csv("./Data/2nd_manipulated_data/20220808_" + str(key)+"_train_minmax.csv")
df_train = df_train.drop(df_train[(df_train[target_species + str('_18')]!=0)&(df_train['Species']!= target_species)].index).reset_index(drop=True)

In [ ]:
df_test = pd.read_csv("./Data/2nd_manipulated_data/20220808_" + str(key)+"_test_minmax.csv")
df_test = df_test[df_test.Species==target_species].reset_index(drop=True)

In [ ]:
#remain columns which you need
delivered_package = set(df_train.columns.tolist()) - set(['Source','State','Year','Latitude','Longitude','public_positional_accuracy', 
                                                    str(target_species+str('_18'))])

delivered_package = list(delivered_package)

df_train = df_train.sort_values(by='Species').reset_index(drop=True)
df_train_18 = df_train[delivered_package]


df_train_18['target'] = 0
df_train_18.loc[df_train.Species==target_species,'target']=1
df_train_18 = df_train_18.sort_values(by='target', ascending=False).reset_index(drop=True)

In [ ]:
df_train_18.rename(columns={"Species":"Species_18"},inplace=True)
df_train_18.columns = [i.replace(" ","_").replace("-","") for i in df_train_18.columns]
    

### AOO 계산

In [ ]:
AOO(df_train = df_train_18,
df_test = df_test ,
targetnum=len(df_test),
varlist=shap_var,Year_start=2007,Year_end=2021,
target_species=target_species,
bagnum=50,repnum=50,learningrate=0.7,to_csv=False)